<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/spm_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
from sklearn.model_selection import train_test_split

2.17.0


In [23]:
# 사용자 정의 신경망 모델을 정의
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# Define the model class
class WeatherANN(tf.keras.Model):
    def __init__(self):
        super(WeatherANN, self).__init__()

        self.hid1 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop1 = tf.keras.layers.Dropout(0.2)

        self.hid2 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop2 = tf.keras.layers.Dropout(0.2)


        self.result = tf.keras.layers.Dense(12,
                                            activation=None,
                                            kernel_initializer=self.weight_initialization(),
                                            bias_initializer=self.weight_initialization())

    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

    def call(self, x, training=False):  # Forward pass
        y1 = self.hid1(x)
        d1 = self.drop1(y1, training=training)
        y2 = self.hid2(d1)
        d2 = self.drop2(y2, training=training)
        logits = self.result(d2)
        sy = tf.nn.softmax(logits)
        return logits, sy

In [24]:
# Loss function
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Optimizer and backward pass
learningRate = 0.001
optimizer = tf.optimizers.Adam(learningRate)

@tf.function
def backward(model, x, y):
    with tf.GradientTape() as grad:
        logits, sy = model(x, training=True)
        loss = cross_entropy_loss(logits, y)
    grads = grad.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Accuracy function
@tf.function
def accuracy(predY, y):
    correct_predictions = tf.equal(tf.argmax(predY, axis=1), tf.argmax(y, axis=1))
    acc = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return acc

In [25]:
# 훈련 데이터 함수
def train_learning(epochs, frequency, trainDataIter, model, XTest, yTest):
    y_loss = []
    for epoch in tqdm(range(epochs)):
        bx, by = next(trainDataIter)
        loss = backward(model, bx, by)
        y_loss.append(loss)
        if epoch % frequency == 0:
            predY, sy = model(bx, training=False)
            acc = accuracy(predY, by)
            print(f"Epoch: {epoch+1} ===> Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    predY, sy = model(XTest, training=False)
    acc = accuracy(predY, yTest)
    print(f"Test Accuracy: {acc:.4f}")
    return y_loss

In [26]:
path = "/content/drive/MyDrive/세종교육/spm_ann/Dataset_SPM_demagnetization.csv"
df = pd.read_csv(path)
df.keys()
X = df[['T_spec', 'Rated_speed_spec', 'Bmr', 'delta', 'm', 'so', 'd0', 'Lmagn',
       'h4', 'h3', 'max_OL_analytical']].values
y = df[['max_OL']].values
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = tf.one_hot(y_train.flatten(), 12)
y_test = tf.one_hot(y_test.flatten(), 12)


train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=X_train.shape[0]).batch(300).repeat()
trainDataIter = iter(train_data)


epochs = 10000
frequency = 150

model = SPMANN()
y_loss = train_learning(epochs, frequency, trainDataIter, model, X_test, y_test)

  0%|          | 1/10000 [00:01<3:38:42,  1.31s/it]

Epoch: 1 ===> Loss: 70.0358, Accuracy: 0.0867


  2%|▏         | 190/10000 [00:02<00:50, 193.11it/s]

Epoch: 151 ===> Loss: 21.0816, Accuracy: 0.2400


  4%|▎         | 365/10000 [00:02<00:31, 309.82it/s]

...

Epoch: 9751 ===> Loss: 0.6473, Accuracy: 0.8633


 99%|█████████▉| 9941/10000 [00:36<00:00, 316.47it/s]

Epoch: 9901 ===> Loss: 0.7079, Accuracy: 0.7867


100%|██████████| 10000/10000 [00:37<00:00, 269.26it/s]


Test Accuracy: 0.8250
